# 整理语义噪声

- 目前只是爬去标签为"语义网"的所有博客

## 爬取内容

- 标题
- 时间
- 分类
- 正文

## 思路

- 提取每一篇博客的链接
- 根据链接提取相应内容


## References
- [Beautiful Soup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## 意外
- 中文版的BeautifulSoup, 兄弟节点, 文档有错误.

In [1]:
# -*- coding: utf-8 -*-

In [121]:
import os
import urllib2
import requests
import html2text
from bs4 import BeautifulSoup

In [131]:
cate_url = u'http://baojie.org/blog/category/%e7%a7%91%e6%8a%80%e6%b0%91%e5%b7%a5/%e5%b7%a5%e7%a8%8b%e5%88%9b%e4%b8%9a/'

In [4]:
r = requests.get(cate_url)

In [5]:
def getBlogUrls(url):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    div_main = soup.find('div', id="main")
    header_list = [header for header in div_main.find_all('h1')]
    link_list = [link.get('href') for header in header_list for link in header.find_all('a')]
    return link_list

In [19]:
def getCategories(footer):
    cate_list = []
    for a in footer.find_all(attrs={'rel': 'tag'}):
        cate_list.append(a.text)
    categories = ', '.join(cate_list)
    return categories

In [106]:
def getContent(url):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    div_main = soup.find('div', id="main")
    header = div_main.article.header
    time = header.time.text
    content = div_main.article.find(attrs={'class': 'entry-content post_content'})
    for s in content('script'):
        s.extract()
    footer = div_main.article.footer
    categories = getCategories(footer)
    content = u'%s' % content
    return header.h1.text, html2text.html2text(content), categories, time

In [107]:
def createUrls(url, n):
    url_list = [url]
    for i in range(2, n):
        url_list.append(str(url)+'page/%s/' % i)
    return url_list

In [151]:
def write_to_md(content, url, d):
    content_format= u'''{header}
---
    
> Categories: {categories}  
> Time: {time}  
> Original url: <{url}>
    
{content}
    '''.format(header=content[0], categories=content[2], time=content[3], content=content[1], url=url)
    file_name = '%s-%s.md'% (content[3], content[0])
    if '/' in file_name:
        file_name = ''.join(file_name.split('/'))
    with open(u'%s/%s'% (d, file_name), 'aw') as f:
        f.write(content_format.encode('utf-8'))
    f.close()

In [109]:
urls_page = createUrls(cate_url, 3)

In [110]:
urls_blog = []
content = None
for url in urls_page:
    urls_blog += getBlogUrls(url)

In [111]:
len(urls_blog)

27

In [116]:
i = 1
file_name_list = []
for url in urls_blog:
    print i
    content = getContent(url)
    file_name_list.append('%s-%s'% (content[3], content[0]))
    write_to_md(content, url)
    i += 1

1
Done!...
2
Done!...
3
Done!...
4
Done!...
5
Done!...
6
Done!...
7
Done!...
8
Done!...
9
Done!...
10
Done!...
11
Done!...
12
Done!...
13
Done!...
14
Done!...
15
Done!...
16
Done!...
17
Done!...
18
Done!...
19
Done!...
20
Done!...
21
Done!...
22
Done!...
23
Done!...
24
Done!...
25
Done!...
26
Done!...
27
Done!...


In [129]:
def create_index_page(file_name_list, cate_name, d):
    with open('%s/%s.md' % (d, cate_name), 'aw') as f:
        for file_name in file_name_list:
            f.write('\n- [%s](%s.md)'% (file_name.encode('utf-8'), file_name.encode('utf-8')))
    print "Create successfully ...!"
    f.close()

In [118]:
create_index_page(file_name_list, content[2])

Create successfully ...!


In [139]:
def main(cate_url, cate_name, n=2):
    d = u'/Users/xpgeng/Memect/Baojie/%s' % cate_name
    os.makedirs(d)
    urls_page = createUrls(cate_url, n)
    urls_blog = []
    content = None
    for url in urls_page:
        urls_blog += getBlogUrls(url)
    print "The number of blogs: %s" %len(urls_blog)
    i = 1
    file_name_list = []
    for url in urls_blog:
        content = getContent(url)
        file_name_list.append('%s-%s'% (content[3], content[0]))
        write_to_md(content, url, d)
        i += 1
    print "Write to Markdown successfully..."
    create_index_page(file_name_list, cate_name, d)

In [132]:
main(cate_url, u'工程创业', 3)

The number of blogs: 27
Write to Markdown successfully...
Create successfully ...!


In [133]:
cate_url = r'http://baojie.org/blog/category/%e7%94%9f%e6%b4%bb/%e9%9a%8f%e6%84%9f/'

In [134]:
main(cate_url, u'随感', 6)

The number of blogs: 94
Write to Markdown successfully...
Create successfully ...!


In [135]:
cate_url = u'http://baojie.org/blog/category/%e6%96%b9%e6%b3%95%e8%ae%ba/'

In [137]:
main(cate_url, u'方法论', 2)

The number of blogs: 6
Write to Markdown successfully...
Create successfully ...!


In [153]:
cate_url = u'http://baojie.org/blog/category/%e6%8d%a3%e6%b5%86%e7%b3%8a/%e4%bf%a1%e6%81%af%e8%ae%ba/'

In [154]:
main(cate_url, u'信息论')

The number of blogs: 18
Write to Markdown successfully...
Create successfully ...!
